# Laboratorio#1: Modelo Booleano de Recuperación de Información

Un Modelo de Recuperación de Información (MRI) es un sistema diseñado para facilitar la búsqueda y extracción de información relevante de grandes bases de datos o colecciones de documentos. Estos modelos son cruciales en áreas como los motores de búsqueda en internet, los sistemas de gestión documental y las bibliotecas digitales. Permiten a los usuarios encontrar información específica basada en criterios de búsqueda variados. Algunos de los modelos más significativos incluyen el modelo booleano, que utiliza operadores lógicos para la búsqueda; y el modelo vectorial, que emplea conceptos de álgebra lineal para relacionar documentos y consultas.

Durante el laboratorio se estará trabajando con el Modelo Booleano. Este de define como:
- **D**: Vectores binarios asociados a los términos indexados.
- **Q**: Expresión booleana sobre los términos indexados, utilizando los operadores: _and_, _or_, _not_.
- **F**: Álgebra booleana y teoría de conjuntos.
- **R**: $sim(d_j, q) = 
		 		\left\{
		 			\begin{array}{ll} 
		 				1 & si \ \exists \overrightarrow{q_{cc}} : \overrightarrow{q_{cc}} \in \overrightarrow{q_{fnd}} \ \wedge \ (\forall t_i :\  g_i(\overrightarrow{d_j}) = g_i(\overrightarrow{q_{cc}}))\\ \\
		 				0 & en \ otro \ caso
		 			\end{array} 
		 		\right.
		 	$

In [ ]:
# Configurando el entorno

# Fuente del corpus
import ir_datasets

# Trabajo con los operadores lógicos
from sympy import sympify, to_dnf, Not, And, Or

# Facilita el trabajo con los términos indexados del corpus
from gensim.corpora import Dictionary

# Funciones útiles auxiliares
from teacher_help import tokenize

### Cranfield

El corpus Cranfield es un conjunto clásico de datos en el campo de la Recuperación de Información compuesto por aproximadamente 1,400 resúmenes de artículos de investigación en aerodinámica. Cada documento en este corpus incluye un título, un resumen conciso del contenido, y en algunos casos, palabras clave y referencias bibliográficas. Acompañando a estos documentos, hay alrededor de 225 consultas de prueba y sus correspondientes juicios de relevancia, proporcionados por expertos, que indican la pertinencia de cada documento para una consulta específica. Este diseño estructurado y su enfoque específico en temas de aerodinámica hacen del corpus una herramienta esencial para evaluar la eficacia de Sistemas de Recuperación de Información (SRI), sirviendo como un modelo estándar para pruebas comparativas y consistentes en esta disciplina.

La variable **dataset**, instancia de la clase **ir_datasets.datasets.base.Dataset**, tiene 3 funciones:
1. _docs_iter()_
2. _queries_iter()_
3. _qrels_iter()_

Pero durante esta clase, solo se trabajará con la primera de ellas. La función **docs_iter()** devuelve un objeto iterable de tuplas de dimensión 5, referentes a los documentos. Los campos de cada tupla son:
  - Identificador (str)
  - Título (str)
  - Texto (str)
  - Autor (str)
  - Referencia bibliográfica (str)


### Ejercicio #1: Cargue en memoria el corpus Cranfield y tokenice cada documentos.
###### Ayuda: Consulte la función `tokenize/1` hecha por los profesores.

In [ ]:
# raise Exception('Not Implemented')

dataset = ir_datasets.load("cranfield")
tokenized_docs = [(doc[0], tokenize(doc[2], ['and', 'or', 'not', '(', ')', '&', '~', '|'])) for doc in dataset.docs_iter()]

### Creando mi primera estructura de datos para un MRI

Los términos invertidos, también conocidos como **índices invertidos**, son estructuras de datos utilizadas en Sistemas de Recuperación de Información (SRI) para asociar términos (o palabras) con los documentos que los contienen. Básicamente, son diccionarios que asignan términos a una lista de documentos en los que aparecen esos términos. Esto permite una búsqueda eficiente de documentos que contienen ciertos términos.
 

### Ejercicio #2: Implemente una función que dado un corpus tokenizado, devuelva los índices invertidos.
Considere que cada documento posee un identificador único.

In [ ]:
def build_inverted_index(data):
    """
    Build an inverted index structure
    
    Args:
    - data : [(str, [str])]
        List of tuples of size 2. The first position contains the document identifier and the second the list of document tokens.

    Return:
    - dict
    
    """
    # raise Exception('Not Implemented')
    
    inverted_index = {}
    
    for doc_id, document in data:
        for word in document:
            if word not in inverted_index:
                inverted_index[word] = set()
            inverted_index[word].add(doc_id)
                
    return inverted_index

build_inverted_index(tokenized_docs)

### Representación de la consulta en FND

La Forma Normal Disyuntiva es una forma estándar de expresar una fórmula lógica booleana como una disyunción de cláusulas, donde cada cláusula es una conjunción de literales (variables booleanas o sus negaciones). Esta representación es útil en la lógica booleana y en la teoría de la computación porque permite realizar operaciones y simplificaciones sobre las expresiones booleanas de manera sistemática. Además, muchas funciones lógicas pueden ser representadas eficientemente en DNF.

### Ejercicio #3: Implemente una función que dada una query booleana, devuelva la FND de la misma.
###### Ayuda #1: Considere usar las funciones `sympify/1` y `to_dnf/2`.
###### Ayuda #2: Considere usar los símbolos `&`, `|` y `~` para sustituir los operadores lógicos `and`, `or` y `not`.

In [ ]:
def query_to_dnf(query):
    """
    Transforms a boolean expression into its disjunctive normal form
    
    Args:
    - query : str
        Boolean query.

    Return:
    - instance of class Or
    
    """
    # raise Exception('Not Implemented')
    
    query = query.lower().replace("and", "&").replace("or", "|").replace("not", "~")

    # Procesar la consulta con spaCy
    tokens = tokenize(query)

    # Añadir '&' donde sea necesario (convenio)
    processed_tokens = []
    for i in range(len(tokens)):
        processed_tokens.append(tokens[i])
        if i < len(tokens) - 1 and tokens[i] not in ['(', '&', '~', '|'] and tokens[i+1] not in [')', '&', '~', '|']:
            processed_tokens.append('&')

    # Unir la cadena
    processed_query = ' '.join(processed_tokens)
    # Convertir a expresión sympy y aplicar to_dnf
    query_expr = sympify(processed_query, evaluate=False)
    query_dnf = to_dnf(query_expr, simplify=True)

    return query_dnf

query = "A AND (B OR NOT C)"
query_dnf = query_to_dnf(query)
print(query_dnf)
# query_dnf

### Ejercicio #4: Implemente la función de similitud del modelo booleano, de forma que dada la consulta y el conjunto de documentos tokenizados la función devuelva aquellos documentos que satifacen a la consulta.
###### Ayuda: Considere usar las funciones `Dictionary/1`, `Dictionary.token2id/0` y `Dictionary.doc2bow/1`.

In [ ]:
def get_matching_docs(query_dnf):
    """
    Similarity function

    Args:
        - query_dnf : Instance of class Or
        Query in FND

    Return:
    - [boolean]. each position matches a document in the corpus. If the value is True then the document satisfies the query, False otherwise.

    """
    # raise Exception('Not Implemented')
  
    # Crear un diccionario a partir de los documentos tokenizados
    dictionary = Dictionary([doc for _, doc in tokenized_docs])
    # Obtener el vocabulario
    vocabulary = list(dictionary.token2id.keys())

    # Convertir los documentos tokenizados a representación de bolsa de palabras
    corpus = [dictionary.doc2bow(doc) for doc in [doc for _, doc in tokenized_docs]]

    # Función para verificar si un documento satisface una componente conjuntiva de la consulta
    def satisfies_conjunctive_component(doc_bow, conjunctive_component):
        doc_terms = set(dict(doc_bow).keys())
        
        for term in conjunctive_component.args:
            # Obtener el identificador del término en el diccionario
            
            if isinstance(term, Not):
                term_id = dictionary.token2id.get(term.args[0].name, -1)
                # Si el término está negado, debe estar ausente en el documento
                if term_id in doc_terms:
                    return False
            else:
                # Si el término no está negado, debe estar presente en el documento
                term_id = dictionary.token2id.get(term.name, -1)
                if term_id not in doc_terms:
                    return False
        return True
    
    matching_documents = []
    for doc_id, doc_bow in enumerate(corpus):
        # Revisar cada componente conjuntiva de la consulta DNF
        if isinstance(query_dnf, And) or isinstance(query_dnf, Or):
            # Si la consulta DNF tiene múltiples términos
            if any(satisfies_conjunctive_component(doc_bow, component) for component in query_dnf.args):
                matching_documents.append(doc_id)
        else:
            # Si la consulta DNF es un solo término o una única conjunción negada
            if satisfies_conjunctive_component(doc_bow, query_dnf):
                matching_documents.append(doc_id)

    return matching_documents

get_matching_docs(query_dnf)  

### Ejercicio #5: ¿Cómo pudiese implementarse la función de similitud usando solamente la consulta en su FND y la estructura de los índices invertidos creada a partir del corpus procesado?